In [355]:
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from pylab import rcParams
import urllib
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from google.colab import files

from sklearn import metrics

In [356]:
np.set_printoptions(precision=4, suppress = True)
%matplotlib inline
rcParams['figure.figsize'] = 7,4
plt.style.use('seaborn-whitegrid')

In [357]:
oportunidades = pd.read_csv('/content/drive/My Drive/Entrenamieto_ECI_2020.csv')
test = pd.read_csv('/content/drive/MyDrive/Test_TP2_Datos_2020-2C.csv')
oportunidades.head()



,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12598,0,None,Person_Name_51,Person_Name_18,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/30/2016,6/13/2016,Person_Name_18,Product_Family_77,Product_Name_99,EUR,0.52,USD,0.58817,5/1/2016,6/30/2016,2016 - 5,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12600,1,39,Person_Name_51,Person_Name_20,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/20/2016,1/15/2016,Person_Name_20,Product_Family_77,Product_Name_100,EUR,0.53,USD,0.59948,1/18/2016,1/20/2016,2016 - 1,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,Source_7,United States,Account_Name_1794,Opportunity_Name_469,2,760,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_4,12/8/2015,None,None,None,None,None,None,NaT,9/30/2016,9/29/2016,Person_Name_8,Product_Family_81,Product_Name_91,USD,0.48,USD,0.48000,1/25/2016,1/25/2016,2016 - 1,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_415,3,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,NaT,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53000,2/1/2018,3/31/2018,2018 - 2,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_851,4,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,6/2/2017,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53000,2/1/2018,2/28/2018,2018 - 2,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None


In [358]:
test.shape

(2551, 51)

In [359]:
oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = oportunidades['Total_Amount']*0.0096
oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = oportunidades['Total_Amount']*1.17
oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = oportunidades['Total_Amount']*0.70
oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = oportunidades['Total_Amount']*1.29
oportunidades.loc[oportunidades['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

Preparo el df

In [360]:
oportunidades_short = oportunidades[['Opportunity_ID','Region', 'Total_Amount', 'Delivery_Year','Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Stage' ]].rename(columns={'Stage': 'Decision'})
oportunidades_short = oportunidades_short[(oportunidades_short['Decision'] == 'Closed Lost') | (oportunidades_short['Decision'] == 'Closed Won')]
oportunidades_short.dropna(inplace=True)
oportunidades_short['Region'] = oportunidades_short.groupby('Region')['Region'].transform('count')
oportunidades_short['Decision'] = np.where(oportunidades_short['Decision'] == 'Closed Won',1,0)
oportunidades_short.replace('Q1', 1, inplace= True)
oportunidades_short.replace('Q2', 10, inplace= True)
oportunidades_short.replace('Q3', 11, inplace= True)
oportunidades_short.replace('Q4', 100, inplace= True)
oportunidades_short.head()

,Opportunity_ID,Region,Total_Amount,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0,4654,6169176.0,2016,10,1,1,0
1,1,4654,56429.1,2016,1,0,0,1
2,2,3904,83865.6,2016,1,0,0,1
3,3,3904,7421881.5,2018,1,0,1,0
4,4,3904,13357192.5,2018,1,0,1,0


In [361]:
X_prime = oportunidades_short.loc[:,('Opportunity_ID','Region','Total_Amount', 'Delivery_Quarter','Pricing, Delivery_Terms_Approved', 'Pricing, Delivery_Terms_Quote_Appr')].values
y = oportunidades_short.loc[:,'Decision'].values


In [362]:
X = preprocessing.scale(X_prime)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state = 17)
#ojo con el futuro

In [363]:
clf = neighbors.KNeighborsClassifier(n_neighbors=4, metric= "manhattan" )
clf.fit(X_train, y_train)
print(clf)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')


con 3-6 eucl va bien

In [364]:
y_espect = y_test
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_espect, y_pred))


              precision    recall  f1-score   support

           0       0.77      0.84      0.80      2378
           1       0.87      0.81      0.84      3174

    accuracy                           0.82      5552
   macro avg       0.82      0.83      0.82      5552
weighted avg       0.83      0.82      0.83      5552



In [365]:
X_test

array([[ 0.7016, -1.3349, -0.1571, -0.4995, -1.1863, -2.0527],
       [-0.8191,  0.8651, -0.1718,  1.7465,  0.843 ,  0.4872],
       [ 1.4925, -1.3349, -0.1506, -0.7241,  0.843 ,  0.4872],
       ...,
       [ 1.4026, -0.4297, -0.1695,  1.7465,  0.843 ,  0.4872],
       [ 1.1811, -0.4297, -0.1711, -0.4745,  0.843 ,  0.4872],
       [-1.5982,  0.585 , -0.0621, -0.4995,  0.843 ,  0.4872]])

In [366]:
y_pred

array([1, 1, 0, ..., 1, 0, 0])

In [367]:
subir = pd.DataFrame()

subir['target'] = y_pred

subir.value_counts()

target
1         2950
0         2602
dtype: int64

In [368]:
test.loc[test['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = test['Total_Amount']*0.0096
test.loc[test['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = test['Total_Amount']*1.17
test.loc[test['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = test['Total_Amount']*0.70
test.loc[test['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

test.loc[test['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = test['Total_Amount']*1.29
test.loc[test['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [369]:
test_short = test[['Opportunity_ID','Region', 'Total_Amount', 'Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr']]
test_short.dropna(inplace=True)
test_short['Region'] = test_short.groupby('Region')['Region'].transform('count')


test_short.replace('Q1', 1, inplace= True)
test_short.replace('Q2', 10, inplace= True)
test_short.replace('Q3', 11, inplace= True)
test_short.replace('Q4', 100, inplace= True)
test_short.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

,Opportunity_ID,Region,Total_Amount,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,10689,838,189820.80,10,1,1
1,10689,838,91990.08,10,1,1
2,10689,838,148069.35,10,1,1
3,10690,838,284731.20,10,1,1
4,10690,838,183980.16,10,1,1


In [370]:
X = preprocessing.scale(test_short)

In [371]:
pred = clf.predict(X)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [372]:
test_short

,Opportunity_ID,Region,Total_Amount,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,10689,838,189820.800,10,1,1
1,10689,838,91990.080,10,1,1
2,10689,838,148069.350,10,1,1
3,10690,838,284731.200,10,1,1
4,10690,838,183980.160,10,1,1
...,...,...,...,...,...,...
2546,12364,838,124740.000,10,1,1
2547,12365,1016,45054.900,100,1,1
2548,12366,1016,100122.000,100,1,1
2549,12367,1016,143220.000,11,1,1


A continuacion, el declive

In [373]:
#subir = pd.DataFrame()
##subir['Opportunity_ID'] = test_short['Opportunity_ID']
#subir['target'] = pred
#subir.set_index('Opportunity_ID', inplace = True)
#subir.reset_index()
#subir.value_counts()

In [374]:
subir = pd.DataFrame()
subir['Opportunity_ID'] = test_short['Opportunity_ID']
subir['target'] = pred
subir.sort_values('Opportunity_ID',inplace = True)
subir = subir.drop_duplicates('Opportunity_ID',keep = 'last')
subir.set_index('Opportunity_ID', inplace = True)
subir

,target
Opportunity_ID,
10689,0
10690,0
10691,0
10692,0
10693,1
...,...
12364,1
12365,0
12366,0


In [375]:
subir.value_counts()

target
0         957
1         610
dtype: int64

In [376]:
csv3 = subir.to_csv('knn_pred_1.csv')
files.download('knn_pred_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>